In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import periodogram

from scipy.optimize import curve_fit

from stellar_stream import StellarStream

In [ ]:
### Load all simulations into StellarStream type and put in a dictionary for easy access.

from pathlib import Path
p = Path('../../data/sample')
graph = []

simulations = {}

for sim_data in p.glob('*.npz'):
    simulation_number = sim_data.name.removeprefix('simdatag').removesuffix('.npz')
    simulations[simulation_number] = (StellarStream.from_simulation(simulation_number, sim_data)
                                      .select("restrict_phi1", phi1_lim=(-100, 0))
                                      .run_analysis(name="detrend", polynomial_fit_degree=4)
                                      .select("restrict_phi2", phi2_lim=3.0)
    )

In [ ]:
simulations['578'].plot_power_spectrum(smooth=True, sigma=1, interpolate=True, interpolation_resolution=2048)
simulations['578'].plot_power_spectrum(smooth=True, sigma=2, interpolate=True, interpolation_resolution=2048)
simulations['578'].plot_power_spectrum(smooth=True, sigma=4, interpolate=True, interpolation_resolution=2048)

In [ ]:
### This is what we did last time

x, density = simulations['578'].density_phi1(bins=40)
freqs_1, ps_1 = periodogram(density/np.sqrt(len(density)), fs=1.0/(x[1]-x[0]), scaling='density')
freqs_2, ps_2 = periodogram(density[::2]/np.sqrt(len(density[::2])), fs=1.0/(x[1]-x[0]), scaling='density')


plt.plot(freqs_1 * len(density), ps_1, label="full data")
plt.plot(freqs_2 * len(density[::2]), ps_2, label="abbreviated data")
plt.legend()

In [ ]:
### This gives the same results

x, density = simulations['578'].density_phi1(bins=40, smooth=False, normalize=False, use_bins=True)
fs = 1.0/(x[1]-x[0])
freqs_1, ps_1 = periodogram(density, fs=fs, scaling='density')
freqs_2, ps_2 = periodogram(density[::2], fs=fs/2, scaling='density')

plt.plot(freqs_1, ps_1, label="full data")
plt.plot(freqs_2, ps_2, label="abbreviated data")
plt.legend()

In [ ]:
### This is what actually happened in the older version of the function, which is what we want

x_high_res, density_high_res = simulations['578'].density_phi1(bins=40, smooth=False, normalize=False, use_bins=True)
x_low_res, density_low_res = simulations['578'].density_phi1(bins=80, smooth=False, normalize=False, use_bins=True)

fs_1 = 1.0/(x_high_res[1]-x_high_res[0])
fs_2 = 1.0/(x_low_res[1]-x_low_res[0])

freqs_1, ps_1 = periodogram(density_high_res, fs=fs_1, scaling='density')
freqs_2, ps_2 = periodogram(density_low_res, fs=fs_2, scaling='density')

plt.plot(freqs_1, ps_1, label="40 bins")
plt.plot(freqs_2, ps_2, label="80 bins")
plt.legend()

In [ ]:
### What the new version of the function does

x_high_res, density_high_res = simulations['578'].density_phi1(bins=40, smooth=False, normalize=False, use_bins=True)
x_low_res, density_low_res = simulations['578'].density_phi1(bins=80, smooth=False, normalize=False, use_bins=True)

fs_1 = 1.0/(x_high_res[1]-x_high_res[0])
fs_2 = 1.0/(x_low_res[1]-x_low_res[0])

freqs_1, ps_1 = periodogram(density_high_res/np.sqrt(len(density_high_res)), fs=fs_1, scaling='density')
freqs_2, ps_2 = periodogram(density_low_res/np.sqrt(len(density_low_res)), fs=fs_2, scaling='density')

plt.plot(freqs_1 * len(density_high_res), ps_1, label="40 bins")
plt.plot(freqs_2 * len(density_low_res), ps_2, label="80 bins")
plt.legend()

In [ ]:
### Rewritten Function

# --- Power spectrum of the φ₁‐density ---------------------------
def power_spectrum(self, precision: float=1.0, bins: int=256, use_bins=False, window='boxcar', detrend=None):
    """
    Function to produce the power spectrum of the phi1 densities.
    Parameters
    ----------
        precision:
            The precision in degrees for binning.
        use_bins:
            If True, uses the provided bins instead of calculating from precision. False by default.
        bins: int
            The number of bins to use for the densities.
        window: None or int
            Window size or None for no window.
        detrend: str or None
            Detrending method to apply before computing the periodogram. Options are 'linear', 'constant', or None.
    Returns
    -------
        frequencies: array
            The frequencies.
        power_spectrum: array
            The power spectral densities.
    """
    
    if not use_bins:
        bins = self.degrees_to_bins(precision)

    key = ("ps", bins, window, detrend, remove_nyquist)
    if key not in self._cache:
        x, dens = self.density_phi1(bins=bins, smooth=False, normalize=False, use_bins=True)
        fs = 1.0/(x[1]-x[0])
        freqs, ps = periodogram(dens/np.sqrt(len(dens)), fs=fs, window=window, detrend=detrend)
        freqs *= len(dens)
            
        self._cache[key] = (freqs, ps)
    
    return self._cache[key]

In [ ]:
from typing import Tuple
from scipy.ndimage import gaussian_filter1d


def density(phi1, phi1_min, phi1_max, bins=100, sigma_bins=2, smooth=True, interpolate=True, interpolation_resolution=4000) -> Tuple[np.ndarray, np.ndarray]:
    """
    Estimate PDF from samples `phi1` over [phi1_min, phi1_max] using `bins` uniform bins.
    
    Return (centers, density).
    sigma_bins is the gaussian_filter1d sigma in bin units (not in phi1 units).
    """
    phi1 = np.asarray(phi1)
    counts, edges = np.histogram(phi1, bins=bins, range=(phi1_min, phi1_max), density=True)
    centers = (edges[:-1] + edges[1:]) / 2.0
    
    if smooth:
        # smooth the density values along the bin axis
        counts = gaussian_filter1d(counts, sigma=sigma_bins, mode='reflect')

        # renormalize so integral(counts) == 1
        widths = np.diff(edges)
        integral = np.sum(counts * widths)
        if integral > 0:
            counts /= integral
        else:
            # no data in range → return zeros (or raise)
            counts = np.zeros_like(counts, dtype=float)

    if interpolate:
        # Interpolate the density onto a finer grid
        centers_fine = np.linspace(phi1_min, phi1_max, interpolation_resolution)
        counts_fine = np.interp(centers_fine, centers, counts, left=0, right=0)
        return centers_fine, counts_fine

    return centers, counts


phi1 = simulations['578'].phi1
dens_high_res = density(phi1, -100, 0, 100, smooth=True, interpolate=True, interpolation_resolution=6000)
dens_low_res = density(phi1[::10], -100, 0, 100, smooth=True, interpolate=True, interpolation_resolution=6000)
plt.plot(dens_high_res[0], dens_high_res[1], label="high res")
plt.plot(dens_low_res[0], dens_low_res[1], label="low res")
plt.legend()

In [ ]:
x_high_res, density_high_res = dens_high_res
x_low_res, density_low_res = dens_low_res

fs_1 = 1.0/(x_high_res[1]-x_high_res[0])
fs_2 = 1.0/(x_low_res[1]-x_low_res[0])

freqs_1, ps_1 = periodogram(density_high_res, fs=fs_1, scaling='density')
freqs_2, ps_2 = periodogram(density_low_res, fs=fs_2, scaling='density')

plt.plot(freqs_1[1:], ps_1[1:], label="high res")
plt.plot(freqs_2[1:], ps_2[1:], label="low res")
plt.xscale("log")
plt.yscale("log")
plt.legend()